In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from metadata import *

import time

import matplotlib.pyplot as plt
%matplotlib inline

Setting environment for Gordon


In [ ]:
# Compute Gradient - score volume

# for stack in ['MD594', 'MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602']:
# for stack in ['MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602']:
for stack in ['MD589']:
    
    print stack
    
    # numpy sparse does not support 3d
    section_bs_begin, section_bs_end = section_range_lookup[stack]

#     (volume_xmin, volume_xmax, volume_ymin, volume_ymax, volume_zmin, volume_zmax) = \
#     np.loadtxt(os.path.join(volume_dir, '%(stack)s/%(stack)s_scoreVolume_limits.txt' % {'stack': stack}), dtype=np.int)
            
    volume2_allLabels = {}

    for name in labels_unsided:

        if name == 'BackG':
            continue

        volume2_roi = bp.unpack_ndarray_file(os.path.join(volume_dir, '%(stack)s/%(stack)s_scoreVolume_%(label)s.bp' % \
                                                          {'stack': stack, 'label': name})).astype(np.float16)
        volume2_allLabels[name] = volume2_roi
        del volume2_roi

#     test_ydim, test_xdim, test_zdim = volume2_allLabels.values()[0].shape
#     test_centroid = np.array([.5*test_xdim, .5*test_ydim, .5*test_zdim])

    t1 = time.time()

    for name in labels_unsided:

        if name == 'BackG':
            continue

        t = time.time()

        gy_gx_gz = np.gradient(volume2_allLabels[name].astype(np.float32), 3, 3, 3) 
        # 3.3 second, much faster than loading

        sys.stderr.write('gradient %s: %f seconds\n' % (name, time.time() - t))

        t = time.time()

#         bp.pack_ndarray_file(gy_gx_gz[0], volume_dir + '/%(stack)s/%(stack)s_scoreVolume_%(label)s_gy.bp' % {'stack':stack, 'label':name})
#         bp.pack_ndarray_file(gy_gx_gz[1], volume_dir + '/%(stack)s/%(stack)s_scoreVolume_%(label)s_gx.bp' % {'stack':stack, 'label':name})
#         bp.pack_ndarray_file(gy_gx_gz[2], volume_dir + '/%(stack)s/%(stack)s_scoreVolume_%(label)s_gz.bp' % {'stack':stack, 'label':name})

        del gy_gx_gz

        sys.stderr.write('save %s: %f seconds\n' % (name, time.time() - t))


    sys.stderr.write('overall: %f seconds\n' % (time.time() - t1))

In [3]:
# Compute Gradient - Annotation Volume

# for stack in ['MD594', 'MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602']:
# for stack in ['MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602']:
# for stack in ['MD594']:
stack = 'MD589'
    
print stack

# numpy sparse does not support 3d
# section_bs_begin, section_bs_end = section_range_lookup[stack]

volume = bp.unpack_ndarray_file(os.path.join(volume_dir, '%(stack)s/%(stack)s_down32_annotationVolume.bp' % \
                                             {'stack': stack}))

MD589


In [4]:
label_indices = set(np.unique(volume)) - {0}

In [5]:
#     volumes_allLabels = {labels_sided[i-1]: np.zeros_like(volume) for i in label_indices}

structure_bboxes = {}
with open(os.path.join(volume_dir, stack, stack+'_down32_annotationVolume_landmarkBboxes.txt'), 'r') as f:
    for ln in f.readlines():
        ln_split = ln.split()
        structure_bboxes[ln_split[0]] = map(int, ln_split[1:]) # xmin,xmax,ymin,ymax,zmin,zmax

In [6]:
name_to_label = {}
label_to_name = {}
with open(os.path.join(volume_dir, stack, stack+'_down32_annotationVolume_nameToLabel.txt'), 'r') as f:
    for line in f.readlines():
        name, label = line.split()
        name_to_label[name] = int(label)
        label_to_name[int(label)] = name

In [7]:
gradient_dir = create_if_not_exists(os.path.join(volume_dir, stack, 'gradients'))

In [8]:
from skimage.filter import gaussian

In [13]:
t1 = time.time()

for i in label_indices:

    name = label_to_name[i]

    xmin, xmax, ymin, ymax, zmin, zmax = structure_bboxes[name]

    mask_inBbox = volume[ymin:ymax+1, xmin:xmax+1, zmin:zmax+1] == i

    volume_oneName = np.zeros((ymax+1-ymin, xmax+1-xmin, zmax+1-zmin), np.float16)
    volume_oneName[mask_inBbox] = 1.
    
    volume_oneName = gaussian(volume_oneName, 3)

    t = time.time()

    try:
        gy_gx_gz_inBox = np.gradient(volume_oneName, 3, 3, 3) 
    # 3.3 second, much faster than loading
    except:
        raise Exception('Error in computing gradient for %s.\n' % name)

    gx = np.zeros_like(volume, dtype=np.float16)
    gx[ymin:ymax+1, xmin:xmax+1, zmin:zmax+1] = gy_gx_gz_inBox[1]
    gy = np.zeros_like(volume, dtype=np.float16)
    gy[ymin:ymax+1, xmin:xmax+1, zmin:zmax+1] = gy_gx_gz_inBox[0]
    gz = np.zeros_like(volume, dtype=np.float16)
    gz[ymin:ymax+1, xmin:xmax+1, zmin:zmax+1] = gy_gx_gz_inBox[2]

    sys.stderr.write('gradient %s: %f seconds\n' % (name, time.time() - t))

    t = time.time()

    bp.pack_ndarray_file(gy, os.path.join(gradient_dir, '%(stack)s_down32_annotationVolume_%(label)s_gy.bp' % {'stack':stack, 'label':name}))
    bp.pack_ndarray_file(gx, os.path.join(gradient_dir, '%(stack)s_down32_annotationVolume_%(label)s_gx.bp' % {'stack':stack, 'label':name}))
    bp.pack_ndarray_file(gz, os.path.join(gradient_dir, '%(stack)s_down32_annotationVolume_%(label)s_gz.bp' % {'stack':stack, 'label':name}))

#     del gy_gx_gz, gx_padded, gy_padded, gz_padded, volume_oneName

    sys.stderr.write('save %s: %f seconds\n' % (name, time.time() - t))


sys.stderr.write('overall: %f seconds\n' % (time.time() - t1))

gradient 10N_L: 0.098471 seconds
save 10N_L: 0.097885 seconds
gradient 10N_R: 0.108572 seconds
save 10N_R: 0.095491 seconds
gradient 12N: 0.111779 seconds
save 12N: 0.098758 seconds
gradient 3N_L: 0.091537 seconds
save 3N_L: 0.097217 seconds
gradient 3N_R: 0.090407 seconds
save 3N_R: 0.095437 seconds
gradient 4N_L: 0.089674 seconds
save 4N_L: 0.094887 seconds
gradient 4N_R: 0.090114 seconds
save 4N_R: 0.093555 seconds
gradient 5N_L: 0.094961 seconds
save 5N_L: 0.097231 seconds
gradient 5N_R: 0.096735 seconds
save 5N_R: 0.096996 seconds
gradient 6N_L: 0.090062 seconds
save 6N_L: 0.092638 seconds
gradient 6N_R: 0.090824 seconds
save 6N_R: 0.093438 seconds
gradient 7N_L: 0.102022 seconds
save 7N_L: 0.097529 seconds
gradient 7N_R: 0.103272 seconds
save 7N_R: 0.098985 seconds
gradient 7n_L: 0.138122 seconds
save 7n_L: 0.099453 seconds
gradient 7n_R: 0.135545 seconds
save 7n_R: 0.100429 seconds
gradient AP: 0.091536 seconds
save AP: 0.099403 seconds
gradient Amb_L: 0.089593 seconds
save Amb_

In [ ]:
# # Compute Gradient - Annotation Volume

# # for stack in ['MD594', 'MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602']:
# # for stack in ['MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602']:
# for stack in ['MD589']:
    
#     print stack
    
#     # numpy sparse does not support 3d
#     section_bs_begin, section_bs_end = section_range_lookup[stack]

#     volume = bp.unpack_ndarray_file(os.path.join(volume_dir, '%(stack)s/%(stack)s_annotationVolume.bp' % \
#                                                  {'stack': stack}))
    
#     label_indices = set(np.unique(volume)) - {0}
            
#     volumes_allLabels = {labels_sided[i-1]: np.zeros_like(volume) for i in label_indices}
#     for i in label_indices:
#         mask = volume == i
#         volumes_allLabels[labels_sided[i-1]][mask] = volume[mask]
#         del mask

#     t1 = time.time()

#     for name in volumes_allLabels.iterkeys():

#         t = time.time()

#         gy_gx_gz = np.gradient(volumes_allLabels[name].astype(np.float32), 3, 3, 3) 
#         # 3.3 second, much faster than loading

#         sys.stderr.write('gradient %s: %f seconds\n' % (name, time.time() - t))

#         t = time.time()

#         bp.pack_ndarray_file(gy_gx_gz[0], volume_dir + '/%(stack)s/%(stack)s_annotationVolume_%(label)s_gy.bp' % {'stack':stack, 'label':name})
#         bp.pack_ndarray_file(gy_gx_gz[1], volume_dir + '/%(stack)s/%(stack)s_annotationVolume_%(label)s_gx.bp' % {'stack':stack, 'label':name})
#         bp.pack_ndarray_file(gy_gx_gz[2], volume_dir + '/%(stack)s/%(stack)s_annotationVolume_%(label)s_gz.bp' % {'stack':stack, 'label':name})

#         del gy_gx_gz

#         sys.stderr.write('save %s: %f seconds\n' % (name, time.time() - t))


#     sys.stderr.write('overall: %f seconds\n' % (time.time() - t1))